# Meta-analysis on log-variance ratio

### Simulation parameters

In [45]:
suppressPackageStartupMessages(library(meta))

Niter <- 1000           # Number of iterations for bootstrap simulation
Nstud <- 10             # Number of studies in the meta-analysis
ss1 <- 20:40             # Different possible number of cases
ss2 <- 5:30             # Different possible number of controls
mu1 <- 10               # True mean for cases
mu2 <- 10               # True mean for controls
sigma1 <- 1.2           # True standard deviation for cases
sigma2 <- 1             # True standard deviation for controls

### Samples generation

In [46]:
n1 <- sample(ss1, Nstud, rep=T)     # Number of cases for each study
n2 <- sample(ss2, Nstud, rep=T)     # Number of controls for each study

ind1 <- lapply(n1, rnorm, mu1, sigma1)      # Cases generation
ind2 <- lapply(n2, rnorm, mu2, sigma2)      # Controls generation
sd1 <- sapply(ind1, sd)                     # Estimated standard deviations for cases of each study
sd2 <- sapply(ind2, sd)                     # Estimated standard deviations for controls of each study

### F-test for comparison two variances

In [47]:
Fr <- sd1^2 / sd2^2                                                 # Fisher ratios
df1 <- n1 - 1                                                       # Degrees of freedom for F-test
df2 <- n2 - 1
f.pvalues <- 1 - abs(pf(Fr, df1, df2) - 0.5) * 2                    # F-test p-values
alpha <- .05
ci <- cbind(Fr/qf(1-alpha/2, df1, df2), Fr/qf(alpha/2, df1, df2))   # 95% Confidence intervals for variance ratios
data.frame(study=1:Nstud, cases=n1, controls=n2, estimate=Fr, lower.95=ci[,1], upper.95=ci[,2], pval=f.pvalues)

study,cases,controls,estimate,lower.95,upper.95,pval
1,20,5,64.2524215,7.4927047,228.655484,0.001044402
2,33,23,1.1478867,0.5086914,2.445758,0.746748103
3,30,24,0.7304647,0.3250354,1.582960,0.420149065
4,35,6,8.9128942,1.4369672,26.453675,0.022244268
5,39,10,2.8526131,0.8117325,7.048691,0.097779012
6,27,6,1.3896578,0.2220454,4.314567,0.770758593
7,32,8,1.5726595,0.3610711,4.293280,0.554542211
8,35,22,1.2524231,0.5494334,2.652331,0.595060851
9,39,6,1.5967615,0.2582386,4.666921,0.638629616
10,29,14,1.5797200,0.5536295,3.805379,0.387683974


### Meta-analysis on ratios of variances

In [48]:
TE <- log(Fr) - digamma(df1/2) + digamma(df2/2) + log(df1/df2)
TEse <- sqrt(trigamma(df1/2) + trigamma(df2/2))
meta <- metagen(TE, TEse)
cat(sprintf("Variance ratio estimate (real value: %g):", sigma1^2/sigma2^2))
with(meta, data.frame(method=c("fixed", "random"), estimate=exp(c(TE.fixed, TE.random)/2),
        lower.95=exp(c(lower.fixed, lower.random)/2),
        upper.95=exp(c(upper.fixed, upper.random)/2), pval=c(pval.fixed, pval.random)))

Variance ratio estimate (real value: 1.44):

method,estimate,lower.95,upper.95,pval
fixed,1.244136,1.055125,1.467007,0.009371519
random,1.384037,1.030134,1.859523,0.031005516


### Monte Carlo simulation

In [49]:
metas <- replicate(Niter, {
	n1 <- sample(ss1, Nstud, rep=T)
	n2 <- sample(ss2, Nstud, rep=T)
	
	ind1 <- lapply(n1, rnorm, mu1, sigma1)
	ind2 <- lapply(n2, rnorm, mu2, sigma2)
	sd1 <- sapply(ind1, sd)
	sd2 <- sapply(ind2, sd)
	
	Fr <- sd1^2 / sd2^2
	df1 <- n1 - 1
	df2 <- n2 - 1
    
	TE1 <- log(Fr) - digamma(df1/2) + digamma(df2/2) + log(df1/df2)     # Correcting for bias
	TE2 <- log(Fr)                                                      # Not correcting for bias
	TEse <- sqrt(trigamma(df1/2) + trigamma(df2/2))
	list(nobias=metagen(TE1, TEse), bias=metagen(TE2, TEse))
}, simplify=F)

### Distribution of estimations

In [50]:
metas1 <- lapply(metas, "[[", "nobias")
metas2 <- lapply(metas, "[[", "bias")
cat(sprintf("Real log-variance ratio: %g\n", 2*log(sigma1/sigma2)))
cat("With bias correction:\n")
TEs1.fixed <- sapply(metas1, "[[", "TE.fixed")
TEs1.random <- sapply(metas1, "[[", "TE.random")
data.frame(mean = c(mean(TEs1.fixed), mean(TEs1.random)), sd=c(sd(TEs1.fixed), sd(TEs1.random)),
            sem = c(sd(TEs1.fixed)/sqrt(length(TEs1.fixed)),
            sd(TEs1.random)/sqrt(length(TEs1.random))), row.names=c("fixed", "random"))
cat("Without bias correction:\n")
TEs2.fixed <- sapply(metas2, "[[", "TE.fixed")
TEs2.random <- sapply(metas2, "[[", "TE.random")
data.frame(mean = c(mean(TEs2.fixed), mean(TEs2.random)), sd=c(sd(TEs2.fixed), sd(TEs2.random)),
            sem = c(sd(TEs2.fixed)/sqrt(length(TEs2.fixed)),
            sd(TEs2.random)/sqrt(length(TEs2.random))), row.names=c("fixed", "random"))


Real log-variance ratio: 0.364643
With bias correction:


,mean,sd,sem
fixed,0.3656070,0.1519189,0.004804097
random,0.3666895,0.1527144,0.004829254


Without bias correction:


,mean,sd,sem
fixed,0.3964108,0.1529670,0.004837243
random,0.3998613,0.1543665,0.004881498


### Power of estimations (rate of log-variance ratios estimated significantly different from 0)

In [51]:
power1.fixed <- mean(sapply(metas1, "[[", "pval.fixed") <= .05)
power1.random <- mean(sapply(metas1, "[[", "pval.random") <= .05)
power2.fixed <- mean(sapply(metas2, "[[", "pval.fixed") <= .05)
power2.random <- mean(sapply(metas2, "[[", "pval.random") <= .05)
data.frame(nobias=c(power1.fixed, power1.random), bias=c(power2.fixed, power2.random), row.names=c("fixed", "random"))

,nobias,bias
fixed,0.686,0.764
random,0.625,0.710


### Frequency of incorrect estimations (Rate of estimated 95% confidence intervals not including real value)

In [52]:
lvr <- 2*log(sigma1/sigma2)
fie1.fixed <- mean(sapply(metas1, "[[", "lower.fixed") > lvr | sapply(metas1, "[[", "upper.fixed") < lvr)
fie1.random <- mean(sapply(metas1, "[[", "lower.random") > lvr | sapply(metas1, "[[", "upper.random") < lvr)
fie2.fixed <- mean(sapply(metas2, "[[", "lower.fixed") > lvr | sapply(metas2, "[[", "upper.fixed") < lvr)
fie2.random <- mean(sapply(metas2, "[[", "lower.random") > lvr | sapply(metas2, "[[", "upper.random") < lvr)
data.frame(nobias=c(fie1.fixed, fie1.random), bias=c(fie2.fixed, fie2.random), row.names=c("fixed", "random"))

,nobias,bias
fixed,0.066,0.073
random,0.049,0.055
